In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 17 13:11:12 2023

@author: Debra Hogue

Modified RankNet by Lv et al. to use Tensorflow not Pytorch
and added additional comments to explain methods

Paper: Simultaneously Localize, Segment and Rank the Camouflaged Objects by Lv et al.
"""

import tensorflow as tf
from tensorflow.keras import activations, layers, losses
import numpy as np
import os, argparse
from datetime import datetime
from Attention.ResNet_models import Generator
from data import get_loader
from utils import adjust_lr, AvgMeter
from scipy import misc
import cv2
from data import test_dataset
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import tensorflow.keras.applications.resnet50 as models # instantiates the ResNet50 architecture 

from utils import l2_regularisation


In [2]:

# parser = argparse.ArgumentParser()
# parser.add_argument('--epoch', type=int, default=1, help='epoch number')
# parser.add_argument('--lr_gen', type=float, default=2.5e-5, help='learning rate for generator')
# parser.add_argument('--batchsize', type=int, default=2, help='training batch size')
# parser.add_argument('--trainsize', type=int, default=480, help='training dataset size')
# parser.add_argument('--decay_rate', type=float, default=0.9, help='decay rate of learning rate')
# parser.add_argument('--decay_epoch', type=int, default=40, help='every n epochs decay learning rate')
# parser.add_argument('--feat_channel', type=int, default=32, help='reduced channel of saliency feat')
# opt = parser.parse_args()
# print('Generator Learning Rate: {}'.format(opt.lr_gen))
epoch = 100

In [3]:

# build models
generator = Generator(channel=32)


# generator_params = generator.parameters()
# generator_optimizer = tf.optimizers.Adam(generator_params, opt.lr_gen)


image_root = './dataset/train/Imgs/'
gt_root = './dataset/train/GT/'
fix_root = './dataset/train/Fix/'

# train_loader = get_loader(image_root, gt_root, fix_root,batchsize=opt.batchsize, trainsize=opt.trainsize)
# total_step = len(train_loader)

CE = losses.BinaryCrossentropy(from_logits=True)
mse_loss = losses.MeanSquaredError()
size_rates = [0.75,1,1.25]  # multi-scale training

def structure_loss(pred, mask):
    padded = tf.pad(mask,tf.constant([[0,0],[15,15],[15,15],[0,0]]))
    pooled =tf.nn.avg_pool2d(padded, ksize=31, strides=1, padding="VALID")
    weit  = 1+5*tf.abs(pooled-mask)
    weit = tf.squeeze(weit)
    wbce= tf.nn.sigmoid_cross_entropy_with_logits(mask,pred)
   
    wbce= tf.math.reduce_mean(wbce)
    wbce  = tf.math.reduce_sum((weit*wbce),axis=[1,2]) /tf.reduce_sum(weit,axis=[1,2])
    mask =tf.squeeze(mask)
    pred  = tf.math.sigmoid(pred)
    pred = tf.squeeze(pred)
    inter = tf.math.reduce_sum((pred*mask)*weit,axis=[1,2])
    union = tf.math.reduce_sum((pred+mask)*weit, axis=[1,2])
    wiou  = 1-(inter+1)/(union-inter+1)
    return tf.math.reduce_mean(wbce+wiou)


        
             
def loss_function(y_true,y_pred):
    gts, fixs = tf.unstack(y_true,2,0)
    gts, _ = tf.split(gts, [1,2], 3)
    fixs, _ = tf.split(fixs, [1,2], 3)
    fix_pred, cod_pred1, cod_pred2 = tf.unstack(y_pred,num=3,axis=0)
    fix_loss = mse_loss(tf.keras.activations.sigmoid(fix_pred),fixs)
    cod_loss1 = structure_loss(cod_pred1, gts)
    cod_loss2 = structure_loss(cod_pred2, gts)
    test= fix_loss + cod_loss1 + cod_loss2
    print("test")
    print(fix_loss)
    print(cod_loss1)
    print(cod_loss2)
    return  fix_loss + cod_loss1 + cod_loss2
    
        
        
        
if __name__ == '__main__':
    
  
    
    
    
    
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch='10, 15')
    
    op= tf.keras.optimizers.Adam(learning_rate=2.5e-5, name='Adam')
    
    generator.compile(optimizer=op, loss=loss_function)
    
    
    data = get_loader(image_root, gt_root, fix_root, 480, 2)
    
  
    generator.fit(x=data,batch_size=2, epochs=epoch, verbose='auto' , callbacks=[tensorboard_callback])
     
    save_path = 'models/Resnet/'
    





    dataset_path = './dataset/test/'



    test_datasets = ['Mine']

Epoch 1/100
test
Tensor("loss_function/mean_squared_error/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("loss_function/Mean_1:0", shape=(), dtype=float32)
Tensor("loss_function/Mean_3:0", shape=(), dtype=float32)
test
Tensor("loss_function/mean_squared_error/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("loss_function/Mean_1:0", shape=(), dtype=float32)
Tensor("loss_function/Mean_3:0", shape=(), dtype=float32)
1000/1000 [==============================] - 554s 535ms/step - loss: 1.9554
Epoch 2/100
1000/1000 [==============================] - 528s 528ms/step - loss: 1.7171
Epoch 3/100
1000/1000 [==============================] - 531s 531ms/step - loss: 1.5472
Epoch 4/100
1000/1000 [==============================] - 531s 531ms/step - loss: 1.4257
Epoch 5/100
1000/1000 [==============================] - 529s 529ms/step - loss: 1.3509
Epoch 6/100
1000/1000 [==============================] - 528s 528ms/step - loss: 1.2676
Epoch 7/100
1000/1000 [===========================

1000/1000 [==============================] - 528s 528ms/step - loss: 0.5215
Epoch 89/100
1000/1000 [==============================] - 530s 530ms/step - loss: 0.5163
Epoch 90/100
1000/1000 [==============================] - 531s 531ms/step - loss: 0.5133
Epoch 91/100
1000/1000 [==============================] - 528s 528ms/step - loss: 0.5244
Epoch 92/100
1000/1000 [==============================] - 529s 529ms/step - loss: 0.5247
Epoch 93/100
1000/1000 [==============================] - 529s 529ms/step - loss: 0.5141
Epoch 94/100
1000/1000 [==============================] - 529s 529ms/step - loss: 0.5075
Epoch 95/100
1000/1000 [==============================] - 531s 531ms/step - loss: 0.5082
Epoch 96/100
1000/1000 [==============================] - 528s 528ms/step - loss: 0.5045
Epoch 97/100
1000/1000 [==============================] - 528s 528ms/step - loss: 0.5112
Epoch 98/100
1000/1000 [==============================] - 529s 528ms/step - loss: 0.5010
Epoch 99/100
1000/1000 [==========

In [ ]:
if not os.path.exists(save_path):
        os.makedirs(save_path)
print("Generator is build")
generator.save("results/model")

In [4]:
for dataset in test_datasets:
    save_path = './results/ResNet50/' + dataset + '/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    image_root = dataset_path + dataset + '/Imgs/'
    test_loader = test_dataset(image_root, 480)

    for i in range(test_loader.size):
        print(i)
        image, HH, WW, name = test_loader.load_data()
        ans = generator(image)
        _,generator_pred, _  = tf.unstack(ans,num=3,axis=0)
        res = generator_pred
        res = tf.image.resize(res, size=tf.constant([WW,HH]), method=tf.image.ResizeMethod.BILINEAR)
        res = tf.math.sigmoid(res).numpy().squeeze()
        res = 255*(res - res.min()) / (res.max() - res.min() + 1e-8)
        print(save_path+name)
        cv2.imwrite(save_path+name, res)
        print()

./dataset/test/Mine/Imgs/
['An_invisibility_cloak_using_optical_camouflage_by_Susumu_Tachi.jpg', 'many things.jpg', 'Peacock_Flounder_Bothus_mancus_in_Kona1.jpg', 'Peacock_Flounder_Bothus_mancus_in_Kona2.jpg', 'Peacock_Flounder_Bothus_mancus_in_Kona3.jpg', 'Peacock_Flounder_Bothus_mancus_in_Kona4.jpg', 'Smallthing big.jpg', 'Untitle.jpg']
8
0
./results/ResNet50/Mine/An_invisibility_cloak_using_optical_camouflage_by_Susumu_Tachi.png

1
./results/ResNet50/Mine/Peacock_Flounder_Bothus_mancus_in_Kona1.png

2
./results/ResNet50/Mine/Peacock_Flounder_Bothus_mancus_in_Kona2.png

3
./results/ResNet50/Mine/Peacock_Flounder_Bothus_mancus_in_Kona3.png

4
./results/ResNet50/Mine/Peacock_Flounder_Bothus_mancus_in_Kona4.png

5
./results/ResNet50/Mine/Smallthing big.png

6
./results/ResNet50/Mine/Untitle.png

7
./results/ResNet50/Mine/many things.png



In [ ]:

#generator2 = Generator(channel=32)
generator2 =  tf.keras.models.load_model('./results/model', custom_objects={'loss_function': loss_function})
for dataset in test_datasets:
    save_path = './results/small/' + dataset + '/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    image_root = dataset_path + dataset + '/Imgs/'
    test_loader = test_dataset(image_root, 480)

    for i in range(test_loader.size):
        print(i)
        image, HH, WW, name = test_loader.load_data()
        ans = generator2(image)
        _,generator_pred, _  = tf.unstack(ans,num=3,axis=0)
        res = generator_pred
        res = tf.image.resize(res, size=tf.constant([WW,HH]), method=tf.image.ResizeMethod.BILINEAR)
        res = tf.math.sigmoid(res).numpy().squeeze()
        res = 255*(res - res.min()) / (res.max() - res.min() + 1e-8)
        print(save_path+name)
        cv2.imwrite(save_path+name, res)
        print()